In [2]:
# Enable module reloading
%load_ext autoreload
%autoreload 2

%load_ext tensorboard

import datetime
import os
import sys
sys.path.append('..')

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import time;

from tensorflow.keras import initializers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, concatenate
from tensorflow.keras.utils import to_categorical
from src.data.load_data import *

In [40]:
df = load_dataset(filename='../data/Jester-Dataset-ratings.csv')
user_ids, joke_ids, ratings = get_data(df, batch_size=10000)
# batch_size = 20000
        
# matrix = df[0:batch_size].pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)
# user_ids = matrix.index.tolist()
# joke_ids = matrix.columns.tolist()
# ratings = matrix.values.flatten().tolist()
# ratings = np.array(cur_df.pivot(index='USER_ID', columns='JOKE_ID', values='Rating').fillna(0)).tolist()

In [41]:
user_onehot = to_categorical(user_ids)
joke_onehot = to_categorical(joke_ids)

In [42]:
len(user_onehot[0])

261

In [43]:
# final_vector = []
# for u, i in zip(user_onehot, joke_onehot):
#     final_vector.append(np.concatenate((u, i), axis=None))

In [44]:
# np.shape(final_vector)|

In [45]:
n_users = len(user_onehot[0]) #len(pd.DataFrame(user_ids)[0].unique())
n_jokes = len(joke_onehot[0]) #len(pd.DataFrame(joke_ids)[0].unique())

In [46]:
print(n_users, n_jokes)

261 151


In [47]:
np.shape(ratings)

(10000,)

In [48]:
print(np.shape(user_onehot), np.shape(np.transpose(user_onehot)))

(10000, 261) (261, 10000)


In [49]:
print(np.shape(joke_onehot), np.shape(np.transpose(joke_onehot)))

(10000, 151) (151, 10000)


In [50]:
[user_onehot, joke_onehot]

[array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]

In [51]:
vector_concat = np.concatenate([user_onehot, joke_onehot], axis=1)

In [52]:
np.array(vector_concat)

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
np.shape(vector_concat)

(10000, 412)

In [54]:
np.shape(vector_concat.T[:n_users].T)

(10000, 261)

In [ ]:
from src.models.model import JokeRecommender

model = JokeRecommender(n_users, n_jokes)

model.compile(
    optimizer='adam', 
    loss='mean_squared_error', 
    metrics=['accuracy']
)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=os.path.join("../logs", str(time.time())),
    histogram_freq=1)

model.fit(
    x=np.array(vector_concat),
    y=np.array(ratings),
    batch_size=100, 
    epochs=50,
    callbacks=[tensorboard_callback],
    validation_split=0.2
)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 6s 703us/sample - loss: 32.9922 - accuracy: 0.0022 - val_loss: 34.3053 - val_accuracy: 0.0015
Epoch 2/50
8000/8000 [==============================] - 5s 607us/sample - loss: 32.8885 - accuracy: 0.0022 - val_loss: 34.3053 - val_accuracy: 0.0015
Epoch 3/50
7900/8000 [============================>.] - ETA: 0s - loss: 32.9106 - accuracy: 0.0022